<a href="https://colab.research.google.com/github/Bmiguez/Arezzo-Valuation/blob/main/Valuation_Arezzo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Importando inicialmente as bibliotecas necessárias
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import sys
import datetime
from dateutil.relativedelta import relativedelta

In [11]:
# Definindo ticker e índice de referencia
ticker = "ARZZ3.SA"
indice = "^BVSP"

# Definindo a data de referência e o intervalo de dados
ano = 2023
mes = 1
dia = 1

data_ref = datetime.date(ano, mes, dia)
data_ini = data_ref - relativedelta(months = 46)

In [12]:
from ast import increment_lineno
# Extrair retornos ajustados da empresa e do benchmark

retornos = yf.download([ticker, indice], start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change()

retornos = pd.DataFrame(retornos.dropna())

y = retornos[ticker]
x = retornos[indice]



[*********************100%***********************]  2 of 2 completed


In [13]:
# Começando a regressão

x = sm.add_constant(x)

modelo = sm.OLS(y, x)

resultados = modelo.fit()

print(resultados.summary())
# Extraindo valor do Beta

beta = resultados.params[1]

conf_interval = resultados.conf_int(alpha=0.05)

lower_bound = conf_interval.iloc[1, 0]
upper_bound = conf_interval.iloc[1, 1]

print("Intervalo de Confiança:")
print("Limite Inferior:", lower_bound)
print("Limite Superior:", upper_bound)


                            OLS Regression Results                            
Dep. Variable:               ARZZ3.SA   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     52.80
Date:                Fri, 23 Jun 2023   Prob (F-statistic):           5.30e-09
Time:                        10:09:03   Log-Likelihood:                 55.153
No. Observations:                  45   AIC:                            -106.3
Df Residuals:                      43   BIC:                            -102.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0106      0.011      0.978      0.3

In [14]:
path = "https://github.com/Bmiguez/Arezzo-Valuation/raw/main/Dados_Valuation.xlsx"
aba_bp = "BP"
aba_dre = "DRE"
aba_fc = "FC"
aba_lp = "LojasProp"
aba_indicadores = "Indicadores"
aba_receitabruta = "Receita Bruta"

#### Criação dos Dataframes
#### Balanço Patrimonial
df_bp = pd.read_excel(path, aba_bp)

#### Demonstração Resultado do Exercício
df_dre = pd.read_excel(path, aba_dre)

#### Fluxo de Caixa
df_fc = pd.read_excel(path, aba_fc)

#### Lojas Próprias
df_lp = pd.read_excel(path, aba_lp)

#### Indicadores
df_ind = pd.read_excel(path, aba_indicadores)

#### Receita Bruta
df_receita = pd.read_excel(path, aba_receitabruta)

In [15]:
# Cálculo do Custo do equity

# Desalavancando beta

tax = 0.27
debt = 1060093
equity = 2716661

# Risk Free Obtido T-bond 5 y 22/06
treasury_nominal = pd.read_csv("https://raw.githubusercontent.com/Bmiguez/Arezzo-Valuation/main/treasury-nominal.csv", delimiter=',', header='infer')
treasury_real = pd.read_csv("https://raw.githubusercontent.com/Bmiguez/Arezzo-Valuation/main/treasury-real.csv", delimiter=',', header='infer')

rf_usd_nominal = treasury_nominal.loc[0:29,["Date", "5 Yr"]].mean()/100
rf_usd_real = treasury_real.loc[0:29,["Date", "5 YR"]].mean()/100

ltn_br = 0.106138 # Vencimento 2026
ntnb = 0.055901 # Vencimento 2026
risco_br = 0.02402083333 # Fonte http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=40940&module=M

# Inflação implicita

cpi_usd = (1 + rf_usd_nominal[0])/(1 + rf_usd_real[0])

cpi_br = (1 + ltn_br)/(1 + ntnb)



# Calculando Retorno de mercado
data_ini = data_ref - relativedelta(years = 5)

retornos = yf.download("^GSPC", start = data_ini, end = data_ref, interval="1mo")["Adj Close"].pct_change().dropna()

retorno_anualizado = (float(retornos.mean())+1)**12-1

#Equity Risk Premium
ERP = (retorno_anualizado - rf_usd_nominal.iloc[0])
ke = rf_usd_nominal[0] + beta * ERP + (cpi_br/cpi_usd -1) + risco_br

# Custo de dívida kd
resultado_fin2022 = df_dre.loc[12,"Resultado Financeiro"]
print(resultado_fin2022)
divida = df_bp.loc[[11,12],["Empréstimos e financiamentos","Empréstimos e financiamentos.1"]].mean(axis=0).sum(axis=0)

kd = -resultado_fin2022/divida

[*********************100%***********************]  1 of 1 completed
-77258


<ipython-input-15-1e6e7a4810ff>:13: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  rf_usd_nominal = treasury_nominal.loc[0:29,["Date", "5 Yr"]].mean()/100
<ipython-input-15-1e6e7a4810ff>:14: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  rf_usd_real = treasury_real.loc[0:29,["Date", "5 YR"]].mean()/100


In [16]:
### Calculando a média das deduções entre receita bruta e liquida
receita_bruta = df_receita.iloc[:,1]
receita_liquida = df_dre.iloc[:,1]

deducoes = pd.DataFrame({"Ano": range(2010,2023), "Percentual":receita_liquida/receita_bruta})

## Calculando a média das deduções nos últimos quatro anos
multiplicador_receita_liq = deducoes.loc[[12,9,8,7,6],"Percentual"].mean()

### Organizando projeções

## Número de lojas

total_FR = df_lp.loc[1:,"Número de franquias"].reset_index(drop=True)
total_LP = df_lp.loc[1:,"Número de lojas próprias"].reset_index(drop=True)
total_mult = df_lp.loc[1:,"Multimarca MI (sem overlap)"].reset_index(drop=True)

abert_FR = df_lp.loc[:,"Número de franquias"].diff().dropna().reset_index(drop=True)
abert_LP = df_lp.loc[:,"Número de lojas próprias"].diff().dropna().reset_index(drop=True)
abert_mult = df_lp.loc[:,"Multimarca MI (sem overlap)"].diff().dropna().reset_index(drop=True)

## Receita por canal
canais = ["Franquias", "Multimarcas", "Lojas próprias", "Web Commerce", "Outros²"]
df_canais = df_receita.loc[:,canais]

receita_LP = df_canais.loc[:,"Lojas próprias"]
receita_FR = df_canais.loc[:,"Franquias"]
receita_mult = df_canais.loc[:,"Multimarcas"]
receita_web = df_canais.loc[:,"Web Commerce"]
receita_out = df_canais.loc[:,"Outros²"]
receita_ext = df_receita.loc[:,"Mercado externo"]

## Receita por unidade

receita_por_LP = receita_LP/ total_LP
receita_por_FR = receita_FR / total_FR
receita_por_mult = receita_mult / total_mult

## Crescimento receita por unidade + receita ecom

var_por_LP = receita_por_LP.pct_change()
var_por_FR = receita_por_FR.pct_change()
var_por_mult = receita_por_mult.pct_change()
var_web = receita_web.pct_change()
var_ext = receita_ext.pct_change()

# Projeções de crescimento de receita

proj_rec_LP = (var_por_LP[12] + var_por_LP[9] + var_por_LP[8] + var_por_LP[7] + var_por_LP[6])/5 + 1
proj_rec_FR = (var_por_FR[12] + var_por_FR[9] + var_por_FR[8] + var_por_FR[7] + var_por_FR[6])/5 + 1
proj_rec_mult = (var_por_mult[12] + var_por_mult[9] + var_por_mult[8] + var_por_mult[7] + var_por_mult[6])/5 + 1
proj_rec_out = (receita_out[12] + receita_out[9] + receita_out[8] + receita_out[7] + receita_out[6])/5 + 1
proj_rec_web = (var_web[12] + var_web[9] + var_web[8] + var_web[7] + var_web[6])/5 + 1
proj_rec_ext = (var_ext[12] + var_ext[9] + var_ext[8] + var_ext[7] + var_ext[6])/5 + 1

# Projeções de número de unidades

proj_abert_FR = (abert_FR[12] + abert_FR[9] + abert_FR[8] + abert_FR[7] + abert_FR[6])/5
proj_abert_LP = (abert_LP[12] + abert_LP[9] + abert_LP[8] + abert_LP[7] + abert_LP[6])/5
proj_abert_mult = (abert_mult[12] + abert_mult[9] + abert_mult[8] + abert_mult[7] + abert_mult[6])/5

1º Valuation

2015-2022

In [17]:
### Primeiro valuation - 2015 até 2022 (ano 2022)

###
i = 12
ano = i + 2010
## Projetando receita bruta (Nomencaltura: 1 = primeiro ano projetado)
# Receita por Loja própria
receita_lp_1 = receita_por_LP[i] * proj_rec_LP #Projetando do ano base
receita_lp_2 = receita_lp_1 * proj_rec_LP
receita_lp_3 = receita_lp_2 * proj_rec_LP
receita_lp_4 = receita_lp_3 * proj_rec_LP
receita_lp_5 = receita_lp_4 * proj_rec_LP

# Total Lojas própriras
total_LP_1 = total_LP[i] + proj_abert_LP #Projetando do ano base
total_LP_2 = total_LP_1 + proj_abert_LP
total_LP_3 = total_LP_2 + proj_abert_LP
total_LP_4 = total_LP_3 + proj_abert_LP
total_LP_5 = total_LP_4 + proj_abert_LP

# Receita por Franquia
receita_fr_1 = receita_por_FR[i] * proj_rec_FR #Projetando do ano base
receita_fr_2 = receita_fr_1 * proj_rec_FR
receita_fr_3 = receita_fr_2 * proj_rec_FR
receita_fr_4 = receita_fr_3 * proj_rec_FR
receita_fr_5 = receita_fr_4 * proj_rec_FR

# Total Franquias
total_FR_1 =  total_FR[i] + proj_abert_FR #Projetando do ano base
total_FR_2 = total_FR_1 + proj_abert_FR
total_FR_3 = total_FR_2 + proj_abert_FR
total_FR_4 = total_FR_3 + proj_abert_FR
total_FR_5 = total_FR_4 + proj_abert_FR

# Receita por multimarca
receita_mult_1 = receita_por_FR[i] * proj_rec_mult #Projetando do ano base
receita_mult_2 = receita_mult_1 * proj_rec_mult
receita_mult_3 = receita_mult_2 * proj_rec_mult
receita_mult_4 = receita_mult_3 * proj_rec_mult
receita_mult_5 = receita_mult_4 * proj_rec_mult

# Total Multimarcas
total_mult_1 =  total_mult[12] + proj_abert_mult #Projetando do ano base
total_mult_2 = total_mult_1 + proj_abert_mult
total_mult_3 = total_mult_2 + proj_abert_mult
total_mult_4 = total_mult_3 + proj_abert_mult
total_mult_5 = total_mult_4 + proj_abert_mult

# Faturanento Web commerce
receita_web_1 = receita_web[i] * proj_rec_web #Projetando do ano base
receita_web_2 = receita_web_1 * (proj_rec_web - 0.025) #Suavizando a estimativa de crescimento para os anos seguintes
receita_web_3 = receita_web_2 * (proj_rec_web - 0.030)
receita_web_4 = receita_web_3 * (proj_rec_web - 0.050)
receita_web_5 = receita_web_4 * (proj_rec_web - 0.100)

# Faturanento Outros

receita_out_1 = receita_out[i] + proj_rec_out #Projetando do ano base
receita_out_2 = receita_out_1 + proj_rec_out
receita_out_3 = receita_out_2 + proj_rec_out
receita_out_4 = receita_out_3 + proj_rec_out
receita_out_5 = receita_out_4 + proj_rec_out

# Faturamento Exterior
receita_ext_1 = receita_ext[i] * proj_rec_ext #Projetando do ano base
receita_ext_2 = receita_ext_1 * proj_rec_ext
receita_ext_3 = receita_ext_2 * proj_rec_ext
receita_ext_4 = receita_ext_3 * proj_rec_ext
receita_ext_5 = receita_ext_4 * proj_rec_ext

## Cálculo da Receita Bruta total

receita_bruta_1 = (receita_lp_1 * total_LP_1) + (receita_fr_1 * total_FR_1) + (receita_mult_1 * total_mult_1) + receita_web_1 + receita_out_1 + receita_ext_1
receita_bruta_2 = (receita_lp_2 * total_LP_2) + (receita_fr_2 * total_FR_2) + (receita_mult_2 * total_mult_2) + receita_web_2 + receita_out_2 + receita_ext_2
receita_bruta_3 = (receita_lp_3 * total_LP_3) + (receita_fr_3 * total_FR_3) + (receita_mult_3 * total_mult_3) + receita_web_3 + receita_out_3 + receita_ext_3
receita_bruta_4 = (receita_lp_4 * total_LP_4) + (receita_fr_4 * total_FR_4) + (receita_mult_4 * total_mult_4) + receita_web_4 + receita_out_4 + receita_ext_4
receita_bruta_5 = (receita_lp_5 * total_LP_5) + (receita_fr_5 * total_FR_5) + (receita_mult_5 * total_mult_5) + receita_web_5 + receita_out_5 + receita_ext_5

receita_projetada = pd.DataFrame({"Ano": [ano + 1, ano + 2, ano + 3, ano + 4, ano + 5], "Receita Bruta":[receita_bruta_1, receita_bruta_2, receita_bruta_3, receita_bruta_4, receita_bruta_5]})
receita_projetada["Receita Liquida"] = receita_projetada["Receita Bruta"] * multiplicador_receita_liq

print(receita_projetada)

### Cálculo do Lucro Bruto
## Markups
df_cmv = df_dre["CMV"]
markup = receita_bruta / (df_cmv * -1)
var_markup = markup.pct_change()
cresc_markup = var_markup.mean() + 1

markup_1 = markup[i] * cresc_markup
markup_2 = markup_1 * cresc_markup
markup_3 = markup_2 * cresc_markup
markup_4 = markup_3 * cresc_markup
markup_5 = markup_4 * cresc_markup

receita_projetada["Markups"] = [markup_1, markup_2, markup_3, markup_4, markup_5]
receita_projetada["CMV"] = receita_projetada["Receita Liquida"] / (receita_projetada["Markups"] * -1)

receita_projetada["Lucro Bruto"] = receita_projetada["Receita Liquida"] + receita_projetada["CMV"]

    Ano  Receita Bruta  Receita Liquida
0  2023   1.744939e+07     1.413060e+07
1  2024   1.984002e+07     1.606654e+07
2  2025   2.257788e+07     1.828367e+07
3  2026   2.568658e+07     2.080111e+07
4  2027   2.911074e+07     2.357400e+07


In [38]:
    ### Projeções de Despesas
    ## Despesas de Lojas próprias e Web Commerce

    desp_lojasecom = df_dre.loc[[6,7,8,9,12],"Lojas próprias e Web Commerce"].sum(axis=0)
    particip_desp_lojasecom = desp_lojasecom / (receita_LP.loc[[6,7,8,9,12]] + receita_web.loc[[6,7,8,9,12]]).sum(axis=0)

    ## Projetando Despesas de Lojas próprias e Web Commerce

    desp_lojasecom_1 = (receita_web_1 + receita_lp_1) * particip_desp_lojasecom
    desp_lojasecom_2 = (receita_web_2 + receita_lp_2) * particip_desp_lojasecom
    desp_lojasecom_3 = (receita_web_3 + receita_lp_3) * particip_desp_lojasecom
    desp_lojasecom_4 = (receita_web_4 + receita_lp_4) * particip_desp_lojasecom
    desp_lojasecom_5 = (receita_web_5 + receita_lp_5) * particip_desp_lojasecom

    ## Despesas de venda, logística e suprimentos

    desp_venda = df_dre.loc[[6,7,8,9,12],"Venda, logística e suprimentos"].sum(axis=0)
    particip_desp_venda = desp_venda / df_receita.loc[[6,7,8,9,12],"Receita bruta total"].sum(axis=0)

    ## Projetando Despesas de venda, logística e suprimentos

    desp_venda_1 = receita_bruta_1 * particip_desp_venda
    desp_venda_2 = receita_bruta_2 * particip_desp_venda
    desp_venda_3 = receita_bruta_3 * particip_desp_venda
    desp_venda_4 = receita_bruta_4 * particip_desp_venda
    desp_venda_5 = receita_bruta_5 * particip_desp_venda

    ## G&A
    desp_gea= df_dre.loc[[6,7,8,9,12],"Despesas gerais e administrativas"].sum(axis=0)
    particip_gea = desp_gea / df_receita.loc[[6,7,8,9,12],"Receita bruta total"].sum(axis=0)

    desp_gea_1 = receita_bruta_1 * particip_gea
    desp_gea_2 = receita_bruta_2 * particip_gea
    desp_gea_3 = receita_bruta_3 * particip_gea
    desp_gea_4 = receita_bruta_4 * particip_gea
    desp_gea_5 = receita_bruta_5 * particip_gea



-0.1344152842656497
